In [1]:
import cv2
import numpy as np

In [2]:
prompt = ["TL ", "BL ", "TR ", "BR "]
points = []
src_field = np.array([[4,3],[5,380],[697,2],[699,380]])
count = 0

In [3]:
# Click event to select corners and a wrapper to clean the code a bit
def click_event(event, x, y, flags, params):
    global points,prompt,count,img,img_cp
    if event == cv2.EVENT_LBUTTONDOWN:
        if count == 4:
            print(points)
            return
        points.append([x,y])
        count += 1
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_cp,prompt[count-1] + str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('image', img_cp)
    if event == cv2.EVENT_RBUTTONDOWN:
        points.clear()
        count = 0
        img_cp = np.copy(img)
        cv2.imshow('image',img_cp)

def imshow_wrap(img):
    cv2.imshow('image',img)
    cv2.setMouseCallback('image',click_event)
    cv2.waitKey(0)
    cv2.destroyAllWindows()    
    
def findIntersection(x1,y1,x2,y2,x3,y3,x4,y4):
    px= ( (x1*y2-y1*x2)*(x3-x4)-(x1-x2)*(x3*y4-y3*x4) ) / ( (x1-x2)*(y3-y4)-(y1-y2)*(x3-x4) ) 
    py= ( (x1*y2-y1*x2)*(y3-y4)-(y1-y2)*(x3*y4-y3*x4) ) / ( (x1-x2)*(y3-y4)-(y1-y2)*(x3-x4) )
    return px,py

In [79]:
# Run this cell only if you want to test with a different source for field image.
# In case the corners are on borders, the selecting order is TL, BL, TR, BR (T: top, L: left..)
img = cv2.imread('img3.jpeg',1)
img_cp = np.copy(img)
imshow_wrap(img_cp)
# points[0][0],points[0][1] = findIntersection(points[0][0],points[0][1],points[1][0],points[1][1],points[2][0],points[2][1],points[3][0],points[3][1])
print(points)
src_field = np.array([[e[0], e[1]] for e in points])
points = []
count = 0

[[451, 242], [209, 470], [1166, 387], [1108, 681]]
[[308, 744], [750, 883], [718, 423], [1029, 472]]


In [4]:
img = cv2.imread('img6.jpeg',1)
img_cp = np.copy(img)
imshow_wrap(img_cp)
# points[0][0],points[0][1] = findIntersection(points[0][0],points[0][1],points[1][0],points[1][1],points[2][0],points[2][1],points[3][0],points[3][1])
print(points)
dst_field = np.array([[e[0], e[1]] for e in points])

[[534, 509], [490, 815], [1154, 515], [1219, 812]]


In [5]:
H,_ = cv2.findHomography(dst_field,src_field)
print(H)
height, width = img.shape[:2]
out = cv2.warpPerspective(img,H,(width,height))
cv2.imshow('transformed',out)
# cv2.imwrite('transformed.jpg',out)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 1.46646341e+00  2.19229065e-01 -8.89257001e+02]
 [-2.13460077e-02  1.96603879e+00 -9.85248436e+02]
 [-7.42700555e-05  7.76373006e-04  1.00000000e+00]]


In [6]:
# This cell is only for calculating Ax = B problem.
# It is independent from the rest of the program.
def p_gen(x,y,xp,yp):
    return [[-x,-y,-1,0,0,0,x*xp,y*xp,xp],[0,0,0,-x,-y,-1,x*yp,y*yp,yp]]

a = p_gen(306,744,4,3)
a += p_gen(753,881,5,380)
a += p_gen(723,421,697,2)
a += p_gen(1026,470,699,380)
b = [0,0,0,0,0,0,0,0,1]
a.append(b)
print(*a,sep="\n")
print("\nH=",np.linalg.solve(np.array(a),np.array(b)).reshape(3,3))

[-306, -744, -1, 0, 0, 0, 1224, 2976, 4]
[0, 0, 0, -306, -744, -1, 918, 2232, 3]
[-753, -881, -1, 0, 0, 0, 3765, 4405, 5]
[0, 0, 0, -753, -881, -1, 286140, 334780, 380]
[-723, -421, -1, 0, 0, 0, 503931, 293437, 697]
[0, 0, 0, -723, -421, -1, 1446, 842, 2]
[-1026, -470, -1, 0, 0, 0, 717174, 328530, 699]
[0, 0, 0, -1026, -470, -1, 389880, 178600, 380]
[0, 0, 0, 0, 0, 0, 0, 0, 1]

H= [[ 2.42228935e+00 -7.79426426e+00  5.09056059e+03]
 [ 5.77342962e+00  7.49826603e+00 -7.32074300e+03]
 [ 4.95593322e-05  9.67334062e-03  1.00000000e+00]]
